In [ ]:
import torch
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())
print(torch.__version__)

In [ ]:
## 모델 환경 설정

from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="nsmc",
    downstream_model_dir="nlpbook/doccls",
    downstream_corpus_root_dir="content/Korpora",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=128,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

In [ ]:
## 랜덤 시드 고정

from ratsnlp import nlpbook
nlpbook.set_seed(args)

In [ ]:
## 로거 설정

nlpbook.set_logger(args)

In [ ]:
## 말뭉치 내려받기

from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True,
)

In [ ]:
## 토크나이저 준비

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

In [ ]:
## 학습 데이터셋 구축
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

In [ ]:
len(train_dataset)

In [ ]:
train_dataset[0]

In [ ]:
## 학습 데이터 로더 구축

from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    # 샘플링 방식을, RandomSampler을 이용하여 batch_size 개수만큼 비복원(replacment=False) 추출
    sampler=RandomSampler(train_dataset, replacement=False),
    # 뽑은 인스턴스를 배치로 만드는 역할
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

In [ ]:
## 평가용 데이터 로드 구축

# 인스턴스를 batch_size만큼 순서대로 추출하는 역할
from torch.utils.data import SequentialSampler
val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)


In [ ]:
## 모델 초기화
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

In [ ]:
## TASK 정의
'''
파이토치 라이트닝이 제공하는 LightningModule 클래스를 상속받아 태스크 정의
태스크에는 모델, 옵티마이저, 학습 과정 등이 정의되어 있음
ClassificationTask에는 옵티마이저, 러닝레이트 스케줄러가 정의되어 있는데,
아담과, ExponentialLR을 사용함
'''

from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

In [ ]:
## 트레이너 정의

trainer = nlpbook.get_trainer(args)

In [15]:
## 학습 캐시

trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)